In [ ]:
example_document = '''Baidu's Apollo Project is one of the world's leading autonomous driving and AI programs, with one of the largest partner ecosystems and over 100 global partners as of 2018, including BYD, Dongfeng, Microsoft, Intel, Nvidia, Daimler AG, ZTE, Grab, Ford, Hyundai and Honda.'''

# GATE

In [ ]:
import requests
url = "https://cloud-api.gate.ac.uk/process-document/annie-named-entity-recognizer"
headers = {'Content-Type': 'text/plain'}
response = requests.post(url, data=example_document, headers=headers).json()

import json
print(json.dumps(response, indent=2))

In [ ]:
def gate_ner(sentence):
  import requests
  return [(sentence[entity["indices"][0]:entity["indices"][1]] + f" ({entity['gender']})",entity_type) if entity_type == "Person" and "gender" in entity else (sentence[entity["indices"][0]:entity["indices"][1]],entity_type)  for entity_type,entities in requests.post("https://cloud-api.gate.ac.uk/process-document/annie-named-entity-recognizer", data=sentence, headers={'Content-Type': 'text/plain'}).json()["entities"].items() for entity in entities]

# https://medium.com/@b.terryjack/nlp-pretrained-named-entity-recognition-7caa5cd28d7b

In [ ]:
gate_ner(example_document)

# NLTK

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

In [ ]:
def nltk_ner(document):
  return {(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

In [ ]:
nltk_ner(example_document)

# Spacy

In [ ]:
!python3 -m spacy download en_core_web_lg

In [ ]:
import spacy
sp_lg = spacy.load('en_core_web_lg') 

In [ ]:
def spacy_large_ner(document):
  return {(ent.text.strip(), ent.label_) for ent in sp_lg(document).ents}

In [ ]:
spacy_large_ner(example_document)

# Flair

In [ ]:
!pip3 install flair

In [ ]:
from flair.models import SequenceTagger
flair_12class = SequenceTagger.load('ner-ontonotes-fast')
flair_4class = SequenceTagger.load('ner')

In [ ]:
def flair_ner(document, model):
  from flair.data import Sentence
  s = Sentence(document)
  model.predict(s)
  entities = s.to_dict(tag_type='ner')
  #print(entities)
  return [(entity["text"], entity["labels"]) for entity in entities["entities"]]

In [ ]:
flair_ner(example_document, flair_4class)

In [ ]:
flair_ner(example_document, flair_12class)

# Deep Pavlov

In [ ]:
!pip3 install deeppavlov
!python3 -m deeppavlov install ner_ontonotes

In [ ]:
from deeppavlov import configs, build_model
deeppavlov_ner = build_model(configs.ner.ner_ontonotes, download=True)

In [ ]:
def convert_entities(entities):
  ents = set()
  for entity,next_entity in zip(entities,entities[1:] + [(".","O")]):
    word,tag = entity
    if tag != "O":
      ent_position, ent_type = tag.split("-")
      if ent_position == "U":
        ents.add((word,ent_type))
      else:
        if ent_position == "B":
          w = word
        elif ent_position == "I":
          w += " " + word
          if next_entity[1].split("-")[0] != "I":
            ents.add((w,ent_type))
  return ents

def dp_ner(sentence):
  tokens,tags = deeppavlov_ner([sentence])
  return convert_entities([(tok,tg) for token,tag in zip(tokens,tags) for tok,tg in list(zip(token,tag)) ])

In [ ]:
dp_ner(example_document)  

# Stanford Core NLP

In [ ]:
!pip3 install nltk==3.2.4

In [ ]:
!wget http://nlp.stanford.edu/software/stanford-ner-2015-04-20.zip
!unzip stanford-ner-2015-04-20.zip 

In [ ]:
from nltk.tag.stanford import StanfordNERTagger
jar = "stanford-ner-2015-04-20/stanford-ner-3.5.2.jar"
model = "stanford-ner-2015-04-20/classifiers/" 
st_3class = StanfordNERTagger(model + "english.all.3class.distsim.crf.ser.gz", jar, encoding='utf8') 
st_4class = StanfordNERTagger(model + "english.conll.4class.distsim.crf.ser.gz", jar, encoding='utf8') 
st_7class = StanfordNERTagger(model + "english.muc.7class.distsim.crf.ser.gz", jar, encoding='utf8') 

In [ ]:
def stanford_ner(document,model):
  if model == 1:
    return [(entity,tag) for entity,tag in st_3class.tag(document.split()) if tag != "O"]
  elif model == 2:
    return [(entity,tag) for entity,tag in st_4class.tag(document.split()) if tag != "O"]
  elif model == 3:
    return [(entity,tag) for entity,tag in st_7class.tag(document.split()) if tag != "O"]

In [ ]:
stanford_ner(example_document,model=1)

In [ ]:
stanford_ner(example_document,model=2)

In [ ]:
stanford_ner(example_document,model=3)

# Allen NLP

In [ ]:
!pip3 install allennlp

In [ ]:
from allennlp.predictors import Predictor
al = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/fine-grained-ner-model-elmo-2018.12.21.tar.gz")

In [ ]:
def convert_results(allen_results):
  ents = set()
  for word, tag in zip(allen_results["words"], allen_results["tags"]):
    if tag != "O":
      ent_position, ent_type = tag.split("-")
      if ent_position == "U":
        ents.add((word,ent_type))
      else:
        if ent_position == "B":
          w = word
        elif ent_position == "I":
          w += " " + word
        elif ent_position == "L":
          w += " " + word
          ents.add((w,ent_type))
  return ents

def allennlp_ner(document):
  return convert_results(al.predict(sentence=document))

In [ ]:
allennlp_ner(example_document)

# Polyglot

In [ ]:
!pip3 install -U git+https://github.com/aboSamoor/polyglot.git@master
!polyglot download embeddings2.en ner2.en
from polyglot.text import Text

In [ ]:
def polyglot_ner(document):
  return {(' '.join(entity),entity.tag.split('-')[-1]) for entity in Text(document).entities}

In [ ]:
polyglot_ner(example_document)